In [1]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
import_path = "/Users/michalmacbookpro/Downloads/titanic/"
train = pd.read_csv(import_path + "train.csv",index_col="PassengerId")
test = pd.read_csv(import_path + "test.csv",index_col="PassengerId")
submision = pd.read_csv(import_path + "gender_submission.csv")

In [3]:
train[["man","woman"]] = pd.get_dummies(train["Sex"])
train[["1","2","3"]] = pd.get_dummies(train["Pclass"])
train[["c","q","s"]] = pd.get_dummies(train["Embarked"])

def cabinizer(df):
    df["Cabin_L"] = df["Cabin"].str.extract(r"(^\D{0,1})")
    df["Cabin_L"].fillna("U",inplace=True)
    df["Cabin_L"] = df["Cabin_L"].map({"A":"ABC","B":"ABC","C":"ABC","D":"DE","E":"DE","F":"FG","G":"FG","T":"ABC","U":"U"})
    return df

def familiazer(df):
    df["Family"] = df["SibSp"] + df["Parch"]
    df["Family"] = df["Family"]
    return df


def aloniazer(df):
    df["Alone"] = 0
    df.loc[df["Family"]==0,"Alone"] = 1
    return df

cabinizer(train)
familiazer(train)
aloniazer(train)

train[["ABC","DE","FG","U"]] = pd.get_dummies(train["Cabin_L"])

to_drop = ["Pclass","Name","Sex","SibSp","Parch","Ticket","Cabin","Embarked","Cabin_L","Family","Age","2","q","FG","man"]
train.drop(to_drop,inplace=True,axis=1)

mean_fare = np.mean(train["Fare"])
std_fare = np.std(train["Fare"])

train["Fare"] = (train["Fare"] - mean_fare)/std_fare
train.sample(15)

,Survived,Fare,man,woman,1,3,c,s,Alone,ABC,DE,U
PassengerId,,,,,,,,,,,,
696,0,-0.376603,0,1,0,0,0,1,1,0,0,1
337,0,0.692549,0,1,1,0,0,1,0,1,0,0
257,1,0.946246,1,0,1,0,1,0,1,0,0,1
223,0,-0.486337,0,1,0,1,0,1,1,0,0,1
535,0,-0.474005,1,0,0,1,0,1,1,0,0,1
20,1,-0.502949,1,0,0,1,1,0,1,0,0,1
776,0,-0.492378,0,1,0,1,0,1,1,0,0,1
729,0,-0.124920,0,1,0,0,0,1,0,0,0,1
468,0,-0.113846,0,1,1,0,0,1,1,0,0,1


In [4]:
y = train["Survived"]
X = train.drop(["Survived"],axis=1)

In [5]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,stratify=y,random_state=42)

In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
#resample nie je aplikovatelne, je to celkom vyrovnane, potom ako posledne
#bagging
from sklearn.ensemble import BaggingClassifier
#halvin random search
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV
from sklearn.linear_model import LogisticRegression

/opt/anaconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [7]:
rfc = RandomForestClassifier()
dtc = DecisionTreeClassifier()

In [8]:
dtc2 = DecisionTreeClassifier(criterion="entropy",max_depth=8)
ada = AdaBoostClassifier(base_estimator=dtc2,learning_rate=0.2,n_estimators=200)
#
ada.fit(X_train,y_train)
y_pred_0 = ada.predict(X_test)
print(accuracy_score(y_pred_0, y_test))

0.7873134328358209


In [9]:
dtc2 = DecisionTreeClassifier(criterion="gini",max_depth=8)
bag = BaggingClassifier(base_estimator=dtc2,n_estimators=150,n_jobs=-1)
bag.fit(X_train, y_train)
y_pred_bag = bag.predict(X_test)
print(accuracy_score(y_pred_bag,y_test))

0.7985074626865671


In [10]:
param = [{
    "criterion": ['gini', 'entropy'],
    "n_estimators":[200,300,400,500],
    'max_depth': range(4, 20, 2),
    'max_leaf_nodes': range(4, 28, 2),
}]
#
gs_rt = GridSearchCV(rfc, param, cv = 5, n_jobs = -1, verbose = 1)
gs_rt.fit(X_train, y_train)
y_pred = gs_rt.predict(X_test)
print(gs_rt.best_estimator_)
print(accuracy_score(y_pred,y_test))
rt_rs_best = gs_rt.best_estimator_

Fitting 5 folds for each of 768 candidates, totalling 3840 fits
RandomForestClassifier(criterion='entropy', max_depth=6, max_leaf_nodes=24,
                       n_estimators=500)
0.8134328358208955


In [11]:
param = [
    {
        'criterion': ['gini', 'entropy'],
        'max_depth': range(2, 20, 2),
        'max_leaf_nodes': range(4, 28, 2),
    }
]
#
gs_dt = GridSearchCV(dtc, param, cv = 5, n_jobs = -1, verbose = 1,scoring="accuracy")
gs_dt.fit(X_train, y_train)
y_pred = gs_dt.predict(X_test)
print(accuracy_score(y_pred,y_test))
dt_gs_best = gs_dt.best_estimator_
print(gs_dt.best_estimator_)
#print(accuracy_score(y_pred,y_test))

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
0.7947761194029851
DecisionTreeClassifier(max_depth=10, max_leaf_nodes=22)


In [12]:
from sklearn.svm import SVC

svc = SVC()

param = [
    {
        'kernel': ['rbf'], 
        'C': [0.1, 0.3, 1, 2, 3, 4], 
        'gamma': [0.1,0.2,0.3]
    }, 
]

svc = SVC(probability = True)
rs_svc = GridSearchCV(svc, param, cv =5, n_jobs = -1, verbose = 1)
rs_svc.fit(X_train, y_train)
y_pred = rs_svc.predict(X_test)
print(accuracy_score(y_pred,y_test))
svc_best = rs_svc.best_estimator_

Fitting 5 folds for each of 18 candidates, totalling 90 fits
0.7910447761194029


In [13]:
param = [
    {
        'criterion': ['gini', 'entropy'],
        'max_depth': range(2, 20, 2),
        'max_leaf_nodes': range(4, 28, 2),
    }
]
#
hs = HalvingRandomSearchCV(dtc,
            param_distributions=param,
            n_candidates='exhaust',
            resource='n_samples',
            factor=1.3,
            random_state=1,
            n_jobs=-1)
#
hs.fit(X_train,y_train)
y_pred_hs = hs.predict(X_test)
print(accuracy_score(y_pred_hs,y_test))

0.7947761194029851


In [14]:
param = [
    {'n_estimators': range(8, 28, 4), 
     'max_depth': range(24, 48, 4),
     'max_leaf_nodes': range(20, 48, 4),
    }
]
#
et = ExtraTreesClassifier()
gs_et = RandomizedSearchCV(et, param, cv = 5, n_jobs = -1, verbose = 1)
gs_et.fit(X_train, y_train)
y_pred = gs_et.predict(X_test)
print(accuracy_score(y_pred,y_test))
print(gs_et.best_estimator_)
extra_best = gs_et.best_estimator_

Fitting 5 folds for each of 10 candidates, totalling 50 fits
0.7873134328358209
ExtraTreesClassifier(max_depth=24, max_leaf_nodes=20, n_estimators=24)


In [15]:
param_grid={
    'max_depth': range(2, 10, 2),
    'n_estimators': range(26, 46, 2),
    'learning_rate': [0.2, 0.1, 0.03, 0.01]
}
#
xg = XGBClassifier(eval_metric='logloss', n_jobs = -1, use_label_encoder = False)
gs_xg = RandomizedSearchCV(xg, param_grid, cv = 5, n_jobs = -1, verbose = 1)
gs_xg.fit(X_train, y_train)
y_pred = gs_xg.predict(X_test)
print(accuracy_score(y_pred,y_test))
xboo_best = gs_xg.best_estimator_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


/opt/anaconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/opt/anaconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/opt/anaconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/opt/anaconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype inste

0.7947761194029851


In [16]:
lr = LogisticRegression()

lr.fit(X_train,y_train)
y_pred_lr = lr.predict(X_test)
print(accuracy_score(y_pred_lr,y_test))

0.7611940298507462


In [17]:
voting_clf = VotingClassifier(estimators=[
    ("rand_for",rt_rs_best),
    ("dec_tree",dt_gs_best),
    ("svc",svc_best),
    ("extree",extra_best),
    ("xgb",xboo_best),
    ("ada",ada),
    ("bag",bag)],
    voting = 'soft', n_jobs = -1, verbose = 1)
voting_clf.fit(X_train, y_train)
y_pred = voting_clf.predict(X_test)
print(accuracy_score(y_pred,y_test))

[Voting] ................. (2 of 7) Processing dec_tree, total=   0.0s
[Voting] ................... (4 of 7) Processing extree, total=   0.0s
[Voting] ...................... (3 of 7) Processing svc, total=   0.1s


/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[Voting] ...................... (6 of 7) Processing ada, total=   0.2s
[Voting] ...................... (7 of 7) Processing bag, total=   0.2s
[Voting] ...................... (5 of 7) Processing xgb, total=   0.3s
[Voting] ................. (1 of 7) Processing rand_for, total=   0.4s
0.7947761194029851


In [18]:
#test = pd.read_csv(import_path + "test.csv",index_col="PassengerId")

test[["man","woman"]] = pd.get_dummies(test["Sex"])
test[["1","2","3"]] = pd.get_dummies(test["Pclass"])
test[["c","q","s"]] = pd.get_dummies(test["Embarked"])

def cabinizer(df):
    df["Cabin_L"] = df["Cabin"].str.extract(r"(^\D{0,1})")
    df["Cabin_L"].fillna("U",inplace=True)
    df["Cabin_L"] = df["Cabin_L"].map({"A":"ABC","B":"ABC","C":"ABC","D":"DE","E":"DE","F":"FG","G":"FG","T":"ABC","U":"U"})
    return df

def familiazer(df):
    df["Family"] = df["SibSp"] + df["Parch"]
    df["Family"] = df["Family"]
    return df


def aloniazer(df):
    df["Alone"] = 0
    df.loc[df["Family"]==0,"Alone"] = 1
    return df

cabinizer(test)
familiazer(test)
aloniazer(test)

test[["ABC","DE","FG","U"]] = pd.get_dummies(test["Cabin_L"])

to_drop = ["Pclass","Name","Sex","SibSp","Parch","Ticket","Cabin","Embarked","Cabin_L","Family","Age","2","q","FG","man"]
test.drop(to_drop,inplace=True,axis=1)

test["Fare"].fillna(np.mean(test["Fare"]),inplace=True)
mean_fare = np.mean(test["Fare"])
std_fare = np.std(test["Fare"])

test["Fare"] = (test["Fare"] - mean_fare)/std_fare
test.sample(15)


submision["Survived"] = voting_clf.predict(test).astype("int")
submision.to_csv('submission.csv', index = False)